<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 20s 787ms/step - loss: 0.6655 - accuracy: 0.6119 - val_loss: 0.6306 - val_accuracy: 0.8244
Epoch 2/10
26/26 [==============================] - 20s 772ms/step - loss: 0.5938 - accuracy: 0.7311 - val_loss: 0.5808 - val_accuracy: 0.6293
Epoch 3/10
26/26 [==============================] - 20s 758ms/step - loss: 0.5315 - accuracy: 0.7445 - val_loss: 0.5115 - val_accuracy: 0.7463
Epoch 4/10
26/26 [==============================] - 20s 762ms/step - loss: 0.4497 - accuracy: 0.8212 - val_loss: 0.5342 - val_accuracy: 0.7073
Epoch 5/10
26/26 [==============================] - 20s 765ms/step - loss: 0.3857 - accuracy: 0.8431 - val_loss: 0.3534 - val_accuracy: 0.8780
Epoch 6/10
26/26 [==============================] - 20s 757ms/step - loss: 0.3456 - accuracy: 0.8662 - val_loss: 0.2789 - val_accuracy: 0.9415
Epoch 7/10
26/26 [==============================] - 20s 754ms/step - loss: 0.2761 - accuracy: 0.9161 - val_loss: 0.2270 - val_accuracy: 0.9463

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.65.h5
26/26 - 20s - loss: 0.6617 - accuracy: 0.6363 - val_loss: 0.6539 - val_accuracy: 0.5756
Epoch 2/5

Epoch 00002: saving model to weights.02-0.63.h5
26/26 - 20s - loss: 0.6107 - accuracy: 0.7080 - val_loss: 0.6302 - val_accuracy: 0.5610
Epoch 3/5

Epoch 00003: saving model to weights.03-0.51.h5
26/26 - 20s - loss: 0.5536 - accuracy: 0.7506 - val_loss: 0.5104 - val_accuracy: 0.7707
Epoch 4/5

Epoch 00004: saving model to weights.04-0.44.h5
26/26 - 20s - loss: 0.5010 - accuracy: 0.7676 - val_loss: 0.4388 - val_accuracy: 0.8049
Epoch 5/5

Epoch 00005: saving model to weights.05-0.49.h5
26/26 - 20s - loss: 0.4497 - accuracy: 0.7956 - val_loss: 0.4889 - val_accuracy: 0.7756


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6746 - accuracy: 0.5888 - val_loss: 0.6742 - val_accuracy: 0.4927


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6744 - accuracy: 0.6131 - val_loss: 0.6474 - val_accuracy: 0.5951
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6428 - accuracy: 0.6521 - val_loss: 0.7890 - val_accuracy: 0.4341


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6777 - accuracy: 0.5669 - val_loss: 0.6768 - val_accuracy: 0.5171
Epoch 2/50
26/26 - 21s - loss: 0.6420 - accuracy: 0.6387 - val_loss: 0.6156 - val_accuracy: 0.7659
Epoch 3/50
26/26 - 20s - loss: 0.5922 - accuracy: 0.7068 - val_loss: 0.5794 - val_accuracy: 0.6537
Epoch 4/50
26/26 - 20s - loss: 0.5385 - accuracy: 0.7372 - val_loss: 0.7515 - val_accuracy: 0.5122
Epoch 5/50
26/26 - 20s - loss: 0.5243 - accuracy: 0.7530 - val_loss: 0.4868 - val_accuracy: 0.7707
Epoch 6/50
26/26 - 20s - loss: 0.4573 - accuracy: 0.8054 - val_loss: 0.4690 - val_accuracy: 0.8244
Epoch 7/50
26/26 - 20s - loss: 0.3874 - accuracy: 0.8358 - val_loss: 0.3413 - val_accuracy: 0.8976
Epoch 8/50
26/26 - 20s - loss: 0.3631 - accuracy: 0.8491 - val_loss: 0.3110 - val_accuracy: 0.9024
Epoch 9/50
26/26 - 20s - loss: 0.3229 - accuracy: 0.8832 - val_loss: 0.2454 - val_accuracy: 0.9512
Epoch 10/50
26/26 - 20s - loss: 0.2398 - accuracy: 0.9331 - val_loss: 0.1984 - val_accuracy: 0.9415
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 788ms/step - loss: 0.6704 - accuracy: 0.5499 - val_loss: 0.6418 - val_accuracy: 0.6634
Epoch 2/5
26/26 [==============================] - 20s 767ms/step - loss: 0.6029 - accuracy: 0.7397 - val_loss: 0.5608 - val_accuracy: 0.8390
Epoch 3/5
26/26 [==============================] - 19s 750ms/step - loss: 0.5233 - accuracy: 0.7835 - val_loss: 0.4814 - val_accuracy: 0.8098
Epoch 4/5
26/26 [==============================] - 22s 827ms/step - loss: 0.4737 - accuracy: 0.8163 - val_loss: 0.3945 - val_accuracy: 0.8146
Epoch 5/5
26/26 [==============================] - 20s 781ms/step - loss: 0.3993 - accuracy: 0.8358 - val_loss: 0.4154 - val_accuracy: 0.8244


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.549878,0.670667,0.663415,0.641757
1,1,0.739659,0.603106,0.839024,0.560805
2,2,0.783455,0.522945,0.809756,0.481351
3,3,0.816302,0.475333,0.814634,0.394501
4,4,0.835766,0.399577,0.824390,0.415350


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6715 - accuracy: 0.5864 - val_loss: 0.6673 - val_accuracy: 0.5171

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 23s 895ms/step - loss: 0.6394 - accuracy: 0.6740 - val_loss: 0.6420 - val_accuracy: 0.7220

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 804ms/step - loss: 0.6229 - accuracy: 0.7397 - val_loss: 0.6367 - val_accuracy: 0.6732

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 19s 750ms/step - loss: 0.6138 - accuracy: 0.7445 - val_loss: 0.6284 - val_accuracy: 0.7317

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 758ms/step - loss: 0.6090

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 842ms/step - loss: 0.6723 - accuracy: 0.5693 - val_loss: 0.6614 - val_accuracy: 0.5415
Epoch 2/50
26/26 [==============================] - 20s 788ms/step - loss: 0.6344 - accuracy: 0.6642 - val_loss: 0.6087 - val_accuracy: 0.7756
Epoch 3/50
26/26 [==============================] - 20s 761ms/step - loss: 0.5896 - accuracy: 0.7080 - val_loss: 0.5830 - val_accuracy: 0.6780
Epoch 4/50
26/26 [==============================] - 21s 804ms/step - loss: 0.5474 - accuracy: 0.7287 - val_loss: 0.5021 - val_accuracy: 0.7561
Epoch 5/50
26/26 [==============================] - 26s 985ms/step - loss: 0.5127 - accuracy: 0.7749 - val_loss: 0.4432 - val_accuracy: 0.8829
Epoch 6/50
26/26 [==============================] - 21s 827ms/step - loss: 0.4567 - accuracy: 0.8041 - val_loss: 0.4081 - val_accuracy: 0.8244
Epoch 7/50
26/26 [==============================] - 21s 792ms/step - loss: 0.4184 - accuracy: 0.8224 - val_loss: 0.4021 - val_accuracy: 0.7756

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4614), started 0:18:18 ago. (Use '!kill 4614' to kill it.)